In [20]:
import pandas as pd

df = pd.read_excel("data/out/df_de_dci.xlsx")

In [21]:
df.head()

,Unnamed: 0,DCI
0,0,CHLORHYDRATE DE VALACICLOVIR
1,1,FUMARATE DE TENOFOVIR ALAFENAMIDE
2,2,FUMARATE DE TENOFOVIR ALAFENAMIDE
3,3,CHLORHYDRATE DE RILPIVIRINE
4,4,CLAVULANATE DE POTASSIUM


In [23]:
import unidecode
df["DCI"] = (
df["DCI"]
    .astype(str)
    .apply(lambda x: unidecode.unidecode(x))  # enlever les accents
    .str.upper()                              # majuscules
    .str.strip()                              # trim
)


In [24]:
df.shape

(703, 2)

In [25]:
df["DCI"].unique().shape

(458,)

# en code

In [6]:
df.shape

(520, 14)

In [3]:
df_d  = df[["DCI"]]

In [9]:
df_d = (
    df_d
    .assign(DCI=df_d["DCI"].str.split(";"))
    .explode("DCI")
)

In [10]:
df_d.shape

(1073, 2)

In [13]:
df_d=df_d.drop("cip7",axis=1)

In [14]:
df_d

,DCI
0,CHLORHYDRATE DE VALACICLOVIR
1,COBICISTAT
1,ELVITEGRAVIR
1,EMTRICITABINE
1,FUMARATE DE TENOFOVIR ALAFENAMIDE
...,...
518,AUBEPINE
518,BROMURE DE POTASSIUM
518,BROMURE DE SODIUM
518,PASSIFLORE


In [15]:
df_d=df_d[df_d["DCI"].str.contains(r"\bDE\b", na=False)]

In [18]:
df_d.drop_duplicates(subset=["DCI"])

,DCI
0,CHLORHYDRATE DE VALACICLOVIR
1,FUMARATE DE TENOFOVIR ALAFENAMIDE
3,CHLORHYDRATE DE RILPIVIRINE
4,CLAVULANATE DE POTASSIUM
5,FUMARATE DE TENOFOVIR DISOPROXIL
...,...
516,IODURE DE POTASSIUM
516,MOLYBDATE DE SODIUM DIHYDRATE
518,BROMURE DE POTASSIUM
518,BROMURE DE SODIUM


In [19]:
df_d.to_excel("data/out/df_de_dci.xlsx")